Classification using OpenAI API

In [1]:
!pip install datasets openai


You should consider upgrading via the '/Users/ashish/.pyenv/versions/3.10.0/envs/llm/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
from datasets import load_dataset

# Load our data
data = load_dataset("rotten_tomatoes")
data

/Users/ashish/.pyenv/versions/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [16]:
import openai
# Create client
API_KEY = "sk-proj--AAA"
client = openai.OpenAI(api_key=API_KEY)

In [17]:
def chatgpt_generation(prompt, document, model="gpt-3.5-turbo-0125"):
    """Generate an output based on a prompt and an input document."""
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
            },
        {
            "role": "user",
            "content":   prompt.replace("[DOCUMENT]", document)
            }
    ]
    chat_completion = client.chat.completions.create(
      messages=messages,
      model=model,
      temperature=0
    )
    return chat_completion.choices[0].message.content

In [18]:
# Define a prompt template as a base
prompt = """Predict whether the following document is a positive or negative movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""

# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)

'1'

In [25]:
import numpy as np
from tqdm import tqdm

# You can skip this if you want to save your (free) credits
predictions = [chatgpt_generation(prompt, doc) for doc in tqdm(data["test"]["text"][:30])]

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:29<00:00,  1.02it/s]


In [34]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """Create and print the classification report"""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )

    print(performance)

In [35]:
# Extract predictions
y_pred = [int(pred) for pred in predictions]
print(len(y_pred))
# Evaluate performance
evaluate_performance(data["test"]["label"][:30], y_pred)

30
                 precision    recall  f1-score   support

Negative Review       0.00      0.00      0.00         0
Positive Review       1.00      0.77      0.87        30

       accuracy                           0.77        30
      macro avg       0.50      0.38      0.43        30
   weighted avg       1.00      0.77      0.87        30



/Users/ashish/.pyenv/versions/llm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ashish/.pyenv/versions/llm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ashish/.pyenv/versions/llm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
